<a href="https://www.kaggle.com/code/gpreda/prompt-gemma-like-a-pro?scriptVersionId=169447376" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<center><img src="https://res.infoq.com/news/2024/02/google-gemma-open-model/en/headerimage/generatedHeaderImage-1708977571481.jpg" width="400"></center>

# <img src="https://icons.iconarchive.com/icons/microsoft/fluentui-emoji-3d/512/Gem-Stone-3d-icon.png" width="35">Introduction

This Notebook is just for me to check a bit Gemma model. 
Will just load Gemma and demonstrate some very simple use cases.
I intend to learn from this experience so that I can then build someting a bit more complex.

# <img src="https://icons.iconarchive.com/icons/microsoft/fluentui-emoji-3d/512/Gem-Stone-3d-icon.png" width="35">Install Keras NLP and Keras

In [1]:
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.1.1 which is incompatible.


# <img src="https://icons.iconarchive.com/icons/microsoft/fluentui-emoji-3d/512/Gem-Stone-3d-icon.png" width="35">Import packages

In [2]:
import keras
import keras_nlp
import os

# Select the desired backend for Keras. Options: "jax", "tensorflow", or "torch".
os.environ["KERAS_BACKEND"] = "jax"  # Adjust as needed.

# Specific to the JAX backend, this setting helps avoid memory fragmentation, ensuring more efficient resource use.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00"

2024-03-30 14:21:13.119916: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-30 14:21:13.120068: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-30 14:21:13.300600: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# <img src="https://icons.iconarchive.com/icons/microsoft/fluentui-emoji-3d/512/Gem-Stone-3d-icon.png" width="35">Load Gemma Causal LLM through Keras NLP

In [3]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_instruct_2b_en")

Attaching 'config.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.spm' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


# <img src="https://icons.iconarchive.com/icons/microsoft/fluentui-emoji-3d/512/Gem-Stone-3d-icon.png" width="35">Check Gemma parameters

In [4]:
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

# <img src="https://icons.iconarchive.com/icons/microsoft/fluentui-emoji-3d/512/Gem-Stone-3d-icon.png" width="35">Define and run a simple prompt (geography question)

In [5]:
prompt = """
You are an AI assistant designed to answer simple questions.
Please restrict your answer to the exact question asked.
Think step by step, use careful reasoning.
Question: {question}
Answer:
"""
response = gemma_lm.generate(prompt.format(question="What is the surface temperature of the Moon?"), max_length=256)
print(response)

I0000 00:00:1711808628.889877      19 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2024-03-30 14:23:48.943510: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.



You are an AI assistant designed to answer simple questions.
Please restrict your answer to the exact question asked.
Think step by step, use careful reasoning.
Question: What is the surface temperature of the Moon?
Answer:
The surface temperature of the Moon is extremely cold, with an average temperature of -173°C (-283°F).


# <img src="https://icons.iconarchive.com/icons/microsoft/fluentui-emoji-3d/512/Gem-Stone-3d-icon.png" width="35">Try another one (math problem)

In [6]:
prompt = """
You are an AI assistant designed to write simple Python code.
Please answer with the listing of the Python code.
Question: {question}
Answer:
"""
response = gemma_lm.generate(prompt.format(question="Please write a function in Python to calculate the area of a circle of radius r"), max_length=256)
print(response)


You are an AI assistant designed to write simple Python code.
Please answer with the listing of the Python code.
Question: Please write a function in Python to calculate the area of a circle of radius r
Answer:
```python
def area_of_circle(radius):
  """
  Calculates the area of a circle of radius r.

  Args:
    radius: The radius of the circle in meters.

  Returns:
    The area of the circle in square meters.
  """

  # Calculate the area of the circle.
  area = math.pi * radius ** 2

  return area
```


# <img src="https://icons.iconarchive.com/icons/microsoft/fluentui-emoji-3d/512/Gem-Stone-3d-icon.png" width="35">Multiple parameters questions

I create here a prompt with multiple parameters and I ask a variety of questions.

In [7]:
prompt = """
You are an AI assistant designed to answer questions with parameters.
Return the answer formated nicely, for example with bullet points.
Question: What are the best {number} {items} from {place}?
Answer:
"""
response = gemma_lm.generate(prompt.format(
    number="3",
    items="food",
    place="France"
    ), 
    max_length=256)
print(response)


You are an AI assistant designed to answer questions with parameters.
Return the answer formated nicely, for example with bullet points.
Question: What are the best 3 food from France?
Answer:
- Croque monsieur
- Steak frites
- Crème brûlée


In [8]:
response = gemma_lm.generate(prompt.format(
    number="five",
    items="attractions",
    place="Italy"
    ), 
    max_length=256)
print(response)


You are an AI assistant designed to answer questions with parameters.
Return the answer formated nicely, for example with bullet points.
Question: What are the best five attractions from Italy?
Answer:
- The Colosseum
- The Vatican City
- The Trevi Fountain
- The Duomo
- The Uffizi Gallery


In [9]:
response = gemma_lm.generate(prompt.format(
    number="two",
    items="places to retire",
    place="Spain"
    ), 
    max_length=256)
print(response)


You are an AI assistant designed to answer questions with parameters.
Return the answer formated nicely, for example with bullet points.
Question: What are the best two places to retire from Spain?
Answer:
- **Seville**
- **Ibiza**

**Explanation:**

- Seville is a vibrant city with a rich cultural heritage.
- Ibiza is a beautiful island with stunning beaches and a relaxed atmosphere.


# <img src="https://icons.iconarchive.com/icons/microsoft/fluentui-emoji-3d/512/Gem-Stone-3d-icon.png" width="35">Reasoning like Einstein will do

In [10]:
prompt = """
You are a math professor, smart but cool.
Background: A train traveling from Bucharest to Ploiesti (60 km distance) has the speed of 60 km/h. 
The train starts in Bucharest and travels until Ploiesti, once, only in this direction.
A swallow, flying with 90 km/h, fly from Ploiesti to the moving train.
When it reaches the train, the swallow flies back toward Ploiesti,
ahead of the train. At Ploiesti turns again back and continues to fly back and forth 
(between the train approaching Ploiesti and Ploiesti) until the train reaches Ploiesti. 
The swallow will fly continously all the time the train is traveling from Bucharest to Ploiesti.
Reasoning: Think step by step. Explain your reasoning.
Question: {question}
Answer:
"""
response = gemma_lm.generate(prompt.format(question="How many kilometers will travel totally the swallow?"), 
    max_length=512)
print(response)


You are a math professor, smart but cool.
Background: A train traveling from Bucharest to Ploiesti (60 km distance) has the speed of 60 km/h. 
The train starts in Bucharest and travels until Ploiesti, once, only in this direction.
A swallow, flying with 90 km/h, fly from Ploiesti to the moving train.
When it reaches the train, the swallow flies back toward Ploiesti,
ahead of the train. At Ploiesti turns again back and continues to fly back and forth 
(between the train approaching Ploiesti and Ploiesti) until the train reaches Ploiesti. 
The swallow will fly continously all the time the train is traveling from Bucharest to Ploiesti.
Reasoning: Think step by step. Explain your reasoning.
Question: How many kilometers will travel totally the swallow?
Answer:
The swallow will travel a total distance of 120 km. 

**Explanation:**

1. The swallow flies from Ploiesti to the train at 90 km/h. Since the train travels at 60 km/h, the swallow will be flying relative to the train for a total of 

# <img src="https://icons.iconarchive.com/icons/microsoft/fluentui-emoji-3d/512/Gem-Stone-3d-icon.png" width="35">Conclusions


## <img src="https://icons.iconarchive.com/icons/microsoft/fluentui-emoji-3d/512/Gem-Stone-3d-icon.png" width="25">Using gemma_2b_en

Initially, I used the model `gemma_2b_en` and I was not really happy with the results. The main problems I found were:
* The model will answer incorectly to the question
* The model will answer to more question than the one answered
* The model will stop suddently in the middle of a phrase

## <img src="https://icons.iconarchive.com/icons/microsoft/fluentui-emoji-3d/512/Gem-Stone-3d-icon.png" width="25">Using gemma_instruct_2b_en

Then, I got a good advice in the comments from one of the Kagglers (thank you @paultimothymooney) and switched to `gemma_instruct_2b_en` with much better results.

Here is the analysis of results with `gemma_instruct_2b_en`:
* General question: answer to the point, and the answer seems sensible. Well, it is actually not entirely correct, but it is close. And the question did not clarify if it is during solar exposure or in the shadow. 
* Math problem, with answer as Python code: correct, and code is documented.
* Multi-parameters question: all answers are correct. 
* More difficult math problem, requiring reasoning: the answer is not correct, because the reasoning is not correct but, to be fair, ... the problem contains a paradox. Did you (intelligent human reading this) notice what is the paradox?
